# Flow with symlinks

**Requirements**:
-  For Windows user, please creating user permission to [create symbolic links](https://learn.microsoft.com/en-us/windows/security/threat-protection/security-policy-settings/create-symbolic-links).

**Attention**:
- TODO git push/pull
- copy

## 0. Create symbolic links in the flow

In [ ]:
import os
from pathlib import Path

saved_path = os.getcwd()
target_folder = Path("../../flows/standard/flow-with-symlinks")
os.chdir(target_folder)

source_folder = Path("../web-classification")
for file_name in os.listdir(source_folder):
    if not Path(file_name).exists():
        os.symlink(
            source_folder / file_name,
            file_name
        )

os.chdir(saved_path)

## 1. Test flow in local

Execute promptflow CLI to test the flow in local.

In [1]:
! pf flow test --flow ../../flows/standard/flow-with-symlinks

Traceback (most recent call last):
  File "D:\Project\PromptFlow\src\promptflow-sdk\promptflow\utils\generate_tool_meta_utils.py", line 131, in generate_python_tool
    exec(content, m.__dict__)
  File "<string>", line 4, in <module>
ImportError: cannot import name 'tool' from 'promptflow' (unknown location)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\zhrua\Miniconda3\envs\prompt_sdk\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\zhrua\Miniconda3\envs\prompt_sdk\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\zhrua\Miniconda3\envs\prompt_sdk\Scripts\pf.exe\__main__.py", line 7, in <module>
  File "D:\Project\PromptFlow\src\promptflow-sdk\promptflow\_cli\pf.py", line 91, in main
    entry(command_args)
  File "D:\Project\PromptFlow\src\promptflow-sdk\promptflow\_cli\pf.py", line 42, in entry
    dispatch_flow_co

## 2. Create a new batch run in local

### 2.1 Create necessary connections
Connection helps securely store and manage secret keys or other sensitive credentials required for interacting with LLM and other external tools for example Azure Content Safety.

In this notebook, we will use flow `flow-with-symlinks` which uses connection `azure_open_ai_connection` inside, we need to set up the connection if we haven't added it before. After created, it's stored in local db and can be used in any flow.

In [2]:
import promptflow as pf
from promptflow.connections import AzureOpenAIConnection

# client can help manage your runs and connections.
client = pf.PFClient()

try:
    conn_name = "azure_open_ai_connection"
    conn = client.connections.get(name=conn_name)
    print("using existing connection")
except:
    connection = AzureOpenAIConnection(
        name=conn_name,
        api_key="b1bce849b20840948800d228524736c1",
        api_base="https://gpt-test-eus.openai.azure.com/",
        api_type="azure",
        api_version="2023-03-15-preview",
    )

    conn = client.connections.create_or_update(connection)
    print("successfully created connection")

print(conn)

AttributeError: module 'promptflow' has no attribute 'PFClient'

### 2.2 Create a new batch run

In [ ]:
flow = "../../flows/standard/flow-with-symlinks" # path to the flow directory
data = "../../flows/standard/flow-with-symlinks/data.jsonl" # path to the data file

# create batch run with default variant
base_run = pf.run_batch(
    flow=flow,
    data=data
)

pf.stream(base_run)

In [ ]:
details = pf.get_details(base_run)
details.head(10)

## 3. Create a new branch run in azure

### 3.1 Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.


In [ ]:
import json
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
# azure version promptflow apis
import promptflow.azure as pf

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

### 3.2 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# configure global setting pointing to workpsace ml_client
pf.configure(ml_client)

# create the prompt-flow client pointing to azureml workspace
# client = pf.PFClient(ml_client=ml_client)

runtime = "demo-mir" # TODO remove this after serverless ready

### 3.3 Create a new batch run

In [ ]:
flow = "../../flows/standard/flow-with-symlinks" # path to the flow directory
data = "../../flows/standard/flow-with-symlinks/data.jsonl" # path to the data file

# create batch run with default variant
base_run = pf.run_batch(
    flow=flow,
    data=data,
    runtime=runtime,
)

pf.stream(base_run)

In [ ]:
details = pf.get_details(base_run)
details.head(10)